In [61]:
import pandas as pd
import numpy as np
import re
import nltk
from urllib.request import Request, urlopen

In [62]:
df = pd.read_excel('cik_list.xlsx')
df

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt
...,...,...,...,...,...,...
147,12239,SPHERIX INC,200704,2007-04-02,10-K,edgar/data/12239/0001104659-07-024804.txt
148,12239,SPHERIX INC,200705,2007-05-16,NT 10-Q,edgar/data/12239/0001104659-07-040463.txt
149,12239,SPHERIX INC,200705,2007-05-18,10-Q,edgar/data/12239/0001104659-07-041441.txt
150,12239,SPHERIX INC,200705,2007-05-23,10-K/A,edgar/data/12239/0001104659-07-042333.txt


In [63]:
df.dropna(axis=0)

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt
...,...,...,...,...,...,...
147,12239,SPHERIX INC,200704,2007-04-02,10-K,edgar/data/12239/0001104659-07-024804.txt
148,12239,SPHERIX INC,200705,2007-05-16,NT 10-Q,edgar/data/12239/0001104659-07-040463.txt
149,12239,SPHERIX INC,200705,2007-05-18,10-Q,edgar/data/12239/0001104659-07-041441.txt
150,12239,SPHERIX INC,200705,2007-05-23,10-K/A,edgar/data/12239/0001104659-07-042333.txt


In [64]:
df['SECFNAME'] = 'https://www.sec.gov/Archives/' + df['SECFNAME']
df

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...
...,...,...,...,...,...,...
147,12239,SPHERIX INC,200704,2007-04-02,10-K,https://www.sec.gov/Archives/edgar/data/12239/...
148,12239,SPHERIX INC,200705,2007-05-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/12239/...
149,12239,SPHERIX INC,200705,2007-05-18,10-Q,https://www.sec.gov/Archives/edgar/data/12239/...
150,12239,SPHERIX INC,200705,2007-05-23,10-K/A,https://www.sec.gov/Archives/edgar/data/12239/...


In [65]:
add_col = ['mda_positive_score', 'mda_negative_score', 'mda_polarity_score', 'mda_average_sentence_length', 'mda_percentage_of_complex_words',
               'mda_complex_word_count', 'mda_word_count', 'mda_uncertainty_score', 'mda_constraining_score', 'mda_positive_word_proportion', 'mda_negative_word_proportion',
               'mda_uncertainty_word_proportion', 'mda_constraining_word_proportion',
               'qqdmr_positive_score','qqdmr_negative_score', 'qqdmr_polarity_score','qqdmr_average_sentence_length','qqdmr_percentage_of_complex_words','qqdmr_complex_word_count',
               'qqdmr_word_count', 'qqdmr_uncertainty_score', 'qqdmr_constraining_score', 'qqdmr_positive_word_proportion', 'qqdmr_negative_word_proportion',
               'qqdmr_uncertainty_word_proportion', 'qqdmr_constraining_word_proportion', 
               'rf_positive_score', 'rf_negative_score', 'rf_polarity_score', 'rf_average_sentence_length', 'rf_percentage_of_complex_words', 'rf_complex_word_count',
               'rf_word_count', 'rf_uncertainty_score', 'rf_constraining_score', 'rf_positive_word_proportion', 'rf_negative_word_proportion',
               'rf_uncertainty_word_proportion', 'rf_constraining_word_proportion']
add_df1 = pd.DataFrame(columns = add_col)

In [66]:
out_form = pd.concat([df, add_df1])
out_form.columns

Index(['CIK', 'CONAME', 'FYRMO', 'FDATE', 'FORM', 'SECFNAME',
       'mda_positive_score', 'mda_negative_score', 'mda_polarity_score',
       'mda_average_sentence_length', 'mda_percentage_of_complex_words',
       'mda_complex_word_count', 'mda_word_count', 'mda_uncertainty_score',
       'mda_constraining_score', 'mda_positive_word_proportion',
       'mda_negative_word_proportion', 'mda_uncertainty_word_proportion',
       'mda_constraining_word_proportion', 'qqdmr_positive_score',
       'qqdmr_negative_score', 'qqdmr_polarity_score',
       'qqdmr_average_sentence_length', 'qqdmr_percentage_of_complex_words',
       'qqdmr_complex_word_count', 'qqdmr_word_count',
       'qqdmr_uncertainty_score', 'qqdmr_constraining_score',
       'qqdmr_positive_word_proportion', 'qqdmr_negative_word_proportion',
       'qqdmr_uncertainty_word_proportion',
       'qqdmr_constraining_word_proportion', 'rf_positive_score',
       'rf_negative_score', 'rf_polarity_score', 'rf_average_sentence_length

In [67]:
with open('StopWords_GenericLong.txt') as stop_raw:
    stop_words = list(stop_raw)
    stop_words = [i.rstrip() for i in stop_words]
    stop_words = [i.lower() for i in stop_words]

In [68]:
pos_words = pd.read_excel('LoughranMcDonald_SentimentWordLists_2018.xlsx', 'Positive', header = None)
neg_words = pd.read_excel('LoughranMcDonald_SentimentWordLists_2018.xlsx', 'Negative', header = None)

In [69]:
pos_words = pos_words[0]
neg_words = neg_words[0]

pos_scrub = list(set(pos_words)-set(stop_words))
neg_scrub = list(set(neg_words)-set(stop_words))
pos_scrub = [i.lower() for i in pos_scrub]
neg_scrub = [i.lower() for i in neg_scrub]

In [70]:
uncer_dic = pd.read_excel('uncertainty_dictionary.xlsx', header = None)
uncer_words = uncer_dic[0].tolist()
uncer_words = [i.lower() for i in uncer_words]
uncer_words.pop(0)

'word'

In [71]:
cons_dic = pd.read_excel('constraining_dictionary.xlsx', header = None)
cons_words = cons_dic[0].tolist()
cons_words = [i.lower() for i in cons_words]
cons_words.pop(0)

'word'

In [80]:
for x in range(len(out_form)):
    
    url = out_form.iloc[x,5]
    url_data = Request(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'})
    url_data = urlopen(url_data).read().decode('utf-8')
    
    url_data = re.sub("(?is)<table[^>]*>(.*?)<\/table>", "", url_data)
    html_regex = re.compile(r'<.*?>')
    url_data = re.sub(html_regex,'',url_data)
    url_data = url_data.replace('&nbsp;','')
    inp_data = re.sub(r'&#\d+;', '', url_data)
    
    task_1, task_2, task_3 = fetching_data(inp_data)
    task_1, task_2, task_3 = data_clean(task_1, task_2, task_3)
    
    if(task_1 != 0):
        pos_mark1, neg_mark1, pola_mark1 = ponepol(task_1)
        out_form.loc[x, 'mda_positive_score'] = pos_mark1
        out_form.loc[x, 'mda_negative_score'] = neg_mark1
        out_form.loc[x, 'mda_polarity_score'] = pola_mark1
        
        avg_sen_len1 = avg_len(task_1)
        out_form.loc[x, 'mda_average_sentence_length'] = avg_sen_len1
        
        comp_count =0
        wor_count1, comp_count1, percomp_count1 = word_count(task_1, comp_count)
        out_form.loc[x, 'mda_word_count'] = wor_count1
        out_form.loc[x, 'mda_complex_word_count'] = comp_count1
        out_form.loc[x, 'mda_percentage_of_complex_words'] = percomp_count1
        
        uncer_mark1, cons_mark1 = uncer_const(task_1)
        out_form.loc[x, 'mda_uncertainty_score'] = uncer_mark1
        out_form.loc[x, 'mda_constraining_score'] = cons_mark1
        
        per_pos1 = pos_mark1/wor_count1
        per_neg1 = neg_mark1/wor_count1
        out_form.loc[x, 'mda_positive_word_proportion']  = per_pos1
        out_form.loc[x, 'mda_negative_word_proportion']  = per_neg1
    else:
        out_form.loc[x, 6:19] = 0
        
    if(task_2 != 0):
        pos_mark2, neg_mark2, pola_mark2 = ponepol(task_2)
        out_form.loc[x, 'qqdmr_positive_score'] = pos_mark2
        out_form.loc[x, 'qqdmr_negative_score'] = neg_mark2
        out_form.loc[x, 'qqdmr_polarity_score'] = pola_mark2
        
        avg_sen_len2 = avg_len(task_2)
        out_form.loc[x, 'qqdmr_average_sentence_length'] = avg_sen_len2
        
        comp_count = 0
        wor_count2, comp_count2, percomp_count2 = word_count(task_2, comp_count)
        out_form.loc[x, 'qqdmr_word_count'] = wor_count2
        out_form.loc[x, 'qqdmr_complex_word_count'] = comp_count2
        out_form.loc[x, 'qqdmr_percentage_of_complex_words'] = percomp_count2
        
        uncer_mark2, cons_mark2 = uncer_const(task_2)
        out_form.loc[x, 'qqdmr_uncertainty_score'] = uncer_mark2
        out_form.loc[x, 'qqdmr_constraining_score'] = cons_mark2
        
        per_pos2 = pos_mark2/wor_count2
        per_neg2 = neg_mark2/wor_count2
        out_form.loc[x, 'qqdmr_positive_word_proportion']  = per_pos2
        out_form.loc[x, 'qqdmr_negative_word_proportion']  = per_neg2
    else:
        out_form.loc[x, 18:32] = 0
        
    if(task_3 != 0):
        pos_mark3, neg_mark3, pola_mark3 = ponepol(task_3)
        out_form.loc[x, 'rf_positive_score'] = pos_mark3
        out_form.loc[x, 'rf_negative_score'] = neg_mark3
        out_form.loc[x, 'rf_polarity_score'] = pola_mark3
        
        avg_sen_len3 = avg_len(task_3)
        out_form.loc[x, 'rf_average_sentence_length'] = avg_sen_len3
        
        comp_count = 0
        wor_count3, comp_count3, percomp_count3 = word_count(task_3, comp_count)
        out_form.loc[x, 'rf_word_count'] = wor_count3
        out_form.loc[x, 'rf_complex_word_count'] = comp_count3
        out_form.loc[x, 'rf_percentage_of_complex_words'] = percomp_count3
        
        uncer_mark3, cons_mark3 = uncer_const(task_3)
        out_form.loc[x, 'rf_uncertainty_score'] = uncer_mark3
        out_form.loc[x, 'rf_constraining_score'] = cons_mark3
        
        per_pos3 = pos_mark3/wor_count3
        per_neg3 = neg_mark3/wor_count3
        out_form.loc[x, 'rf_positive_word_proportion']  = per_pos3
        out_form.loc[x, 'rf_negative_word_proportion']  = per_neg3
    else:
        out_form.loc[x, 31:44] = 0
    pos_mark1 = neg_mark1 = pola_mark1 = avg_sen_len1 = wor_count1 = comp_count1 = percomp_count1 = uncer_mark1 = cons_mark1 = per_pos1 = per_neg1 = 0
    pos_mark2 = neg_mark2 = pola_mark2 = avg_sen_len2 = wor_count2 = comp_count2 = percomp_count2 = uncer_mark2 = cons_mark2 = per_pos2 = per_neg2 = 0
    pos_mark3 = neg_mark3 = pola_mark3 = avg_sen_len3 = wor_count3 = comp_count3 = percomp_count3 = uncer_mark3 = cons_mark3 = per_pos3 = per_neg3 = 0

In [73]:
def fetching_data(data):
    
    first = "MANAGEMENTS DISCUSSION AND ANALYSIS"
    first_fetch = [x.start() for x in re.finditer(first, data)]
    if(len(first_fetch) != 0):
        first_data = data[first_fetch[0]:]
        first_stop = [x.start() for x in re.finditer('ITEM',first_data)]
        if(len(first_stop) == 0):
            first_data = first_data[0:]
        else:
            first_data = first_data[0:first_stop[0]-1]
    if(len(first_fetch) == 0):
        first_data =0
    
    second = "QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK"
    second_fetch = [x.start() for x in re.finditer(second, data)]
    if(len(second_fetch) != 0):
        second_data = data[second_fetch[0]:]
        second_stop = [x.start() for x in re.finditer('ITEM',second_data)]
        if(len(second_stop) == 0):
            second_data = second_data[0:]
        else:
            second_data = second_data[0:second_stop[0]-1]
    if(len(second_fetch) == 0):
        second_data =0
        
    third = "RISK FACTORS"
    third_fetch = [x.start() for x in re.finditer(third, data)]
    if(len(third_fetch) != 0):
        third_data = data[third_fetch[0]:]
        third_stop = [x.start() for x in re.finditer('ITEM',third_data)]
        if(len(third_stop) == 0):
            third_data = third_data[0:]
        else:
            third_data = third_data[0:third_stop[0]-1]
    if(len(third_fetch) == 0):
        third_data =0
        
    return first_data, second_data, third_data

In [74]:
def data_clean(t1, t2, t3):
    if(t1 != 0):
        t1 = nltk.word_tokenize(t1)
        t1 = [words for words in t1 if words.lower() not in stop_words]
        t1 = ' '.join(t1)
    if(t2 != 0):
        t2 = nltk.word_tokenize(t2)
        t2 = [words for words in t2 if words.lower() not in stop_words]
        t2 = ' '.join(t2)
    if(t3 != 0):
        t3 = nltk.word_tokenize(t3)
        t3 = [words for words in t3 if words.lower() not in stop_words]
        t3 = ' '.join(t3)
        
    return t1, t2, t3

In [75]:
def ponepol(i):
    i = nltk.word_tokenize(i)
    i = [words.lower() for words in i]
    count1 = count2 = 0
    for words in i:
        if words in pos_scrub:
            count1 += 1
        if words in neg_scrub:
            count2 -= 1
    count_l = count2 * -1
    pol_sc = (count1 - count_l)/((count1 + count_l) + 0.000001)
    
    return count1, count2, pol_sc

In [76]:
def avg_len(i):
    num = len(nltk.sent_tokenize(i))
    i = nltk.word_tokenize(i)
    num_w = len(i)
    if(num != 0):
        avg_l = round(num_w/num)
    else:
        avg_l = 0
        
    return avg_l

In [77]:
def word_count(i, com_count):
    i = nltk.word_tokenize(i)
    i = [words.lower() for words in i]
    num_w = len(i)
    com_count = 0                                               
    for x in i:                                           
        vowels = "aeiou"
        if (x.endswith(("es", "ed"))):                             
            count = 0
        else:
            count = 0
            for c in i:
                if (c in vowels):
                    count = count + 1                                      
                if (count > 2):                                              
                    com_count = com_count + 1 
    percom_w = com_count/num_w               
    
    return num_w, com_count, percom_w

In [78]:
def uncer_const(i):
    i = nltk.word_tokenize(i)
    i = [words.lower() for words in i]
    count1 = count2 = 0
    for words in i:
        if words in uncer_words:
            count1 += 1
        if words in cons_words:
            count2 +=1
            
    return count1, count2

In [81]:
out_form

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,...,rf_average_sentence_length,rf_percentage_of_complex_words,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion
0,3662.0,SUNBEAM CORP/FL/,199803.0,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1,3662.0,SUNBEAM CORP/FL/,199805.0,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,3662.0,SUNBEAM CORP/FL/,199808.0,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
3,3662.0,SUNBEAM CORP/FL/,199811.0,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,3662.0,SUNBEAM CORP/FL/,199811.0,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,12239.0,SPHERIX INC,200704.0,2007-04-02,10-K,https://www.sec.gov/Archives/edgar/data/12239/...,0,0,0,0,...,16,0,0,16067,117,107,0.00834008,-0.011452,NaN,NaN
148,12239.0,SPHERIX INC,200705.0,2007-05-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/12239/...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
149,12239.0,SPHERIX INC,200705.0,2007-05-18,10-Q,https://www.sec.gov/Archives/edgar/data/12239/...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
150,12239.0,SPHERIX INC,200705.0,2007-05-23,10-K/A,https://www.sec.gov/Archives/edgar/data/12239/...,0,0,0,0,...,16,0,0,15996,117,106,0.00837709,-0.0115029,NaN,NaN


In [82]:
out_form.to_excel('output.xlsx')